In [1]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings,ChatOllama

In [2]:
model_embedding = OllamaEmbeddings(model="nomic-embed-text:latest")
vectore_store = Chroma(embedding_function=model_embedding,persist_directory="./vectorstore")

In [3]:
vectore_store.get()

{'ids': ['314e0585-6296-45ef-92eb-3aeaa3789d21',
  '2fc6138b-ffc8-4976-a199-9bde4a5fa979',
  'e91a409f-63e1-4f73-a0b2-06c092a59b6d',
  '39836ac2-a92a-4c3a-8b8b-2fd46735032d',
  'a83f4cc5-ad94-4556-a96a-0c5476ecb085',
  'd0b5f6ca-e481-4ffe-8df6-1a75744fc708',
  'c2e670e1-fb41-4ce2-933b-bc8bb515dfb6'],
 'embeddings': None,
 'documents': ["import requests\nimport time\nimport os\nimport logging\nfrom concurrent.futures import ThreadPoolExecutor, as_completed\nfrom typing import List, Optional, Dict\n\n# Initialize logging\nlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')\n\nclass ProxyChecker:\n    def __init__(self, proxy_urls: Dict[str, str], timeout: int = 1, max_retries: int = 3, retry_delay: float = 1.0, max_workers: int = 20):\n        self.proxy_urls = proxy_urls\n        self.timeout = timeout\n        self.max_retries = max_retries\n        self.retry_delay = retry_delay\n        self.max_workers = max_workers\n        self.total_proxies

In [1]:
from typing_extensions import List, TypedDict
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

model_nemo_a = ChatOllama(model="nemotron-mini:latest", keep_alive=True, temperature=0)

NameError: name 'ChatOllama' is not defined

In [5]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    

def retriever(state: State):
    return {"context": vectore_store.similarity_search(state["question"], k=3)}


prompt = PromptTemplate.from_template(
    """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
)

def addDocumentinPromptAndQuery(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model_nemo_a.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retriever, addDocumentinPromptAndQuery])
graph_builder.add_edge(START, "retriever")
graph = graph_builder.compile()

In [6]:
response = graph.invoke({"question": "what is the goal of this project?"})
print(response["answer"])

 The goal of this project is to check and save working proxies for a specific type, such as HTTP or SOCKS4/5, by fetching them from URLs provided in the `proxy_urls` dictionary. It then checks these proxies concurrently using multiple threads (max\_workers=100) and saves any working proxies found to a file named after the proxy type (`proxies/{proxy\_type}.txt`). The project also logs the number of working proxies checked, as well as the URLs from which they were fetched.


In [9]:
response = graph.invoke({"question": "list all python packages/libraries we utilized to build this tool?"})
print(response["answer"])

The Python packages and libraries utilized in building this tool are `requests`, `concurrent.futures`, `logging`, and `os`.


In [10]:
response = graph.invoke({"question": "How we are checking teh proxie? what is the main logic?"})
print(response["answer"])

The main logic of the code is to check if a single proxy is working by sending a request to Google using the `requests` library and checking its status code (200 for success). If successful, it returns the proxy; otherwise, it returns None. The function also handles exceptions related to requests or proxies.
The script then creates an instance of `ProxyChecker`, passing in a dictionary containing proxy URLs (`http`, `socks4`, and `socks5`) from which proxies to check concurrently using a thread pool executor with up to 100 workers. It iterates over the futures returned by the executor, checking if each future's result is not None (i.e., successful), appending it to a list of working proxies (`working_proxy_list`).
Finally, the script saves the working proxies in a file using `open()` and writes them with a newline character at the end. If an OSError occurs while writing to the file, it logs an error message. The total number of proxies checked is logged along with the count of working 